In [2]:
import requests
import pandas as pd
import datetime
import time
import os

def fetch_uniswap_v2_pools(start_date, end_date, api_key, output_file="uniswap_v2_pool_count_timeseries.csv"):
    """
    Fetch the daily count of Uniswap V2 pools and their currency pairs from The Graph API.
    :param start_date: Start date for the time series (YYYY-MM-DD)
    :param end_date: End date for the time series (YYYY-MM-DD)
    :param api_key: Your API key for The Graph
    :param output_file: Path to the output CSV file
    :return: Pandas DataFrame containing the daily pool counts and currency pairs.
    """
    url = f"https://gateway.thegraph.com/api/{api_key}/subgraphs/id/A3Np3RQbaBA6oKJgiwDJeo5T3zrYfGHPWFYayMwtNDum"
    query_template = """
    {{
      pairs(first: 1000, where: {{ createdAtBlockNumber_lte: {block_number} }}) {{
        id
        token0 {{
          symbol
        }}
        token1 {{
          symbol
        }}
      }}
      uniswapFactory(id: "0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f", block: {{ number: {block_number} }}) {{
        pairCount
      }}
    }}
    """

    # Check if output file exists and load existing data
    if os.path.exists(output_file):
        existing_data = pd.read_csv(output_file)
        processed_dates = set(existing_data['date'])
    else:
        existing_data = pd.DataFrame(columns=['date', 'pool_count', 'currency_pairs'])
        processed_dates = set()

    current_date = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.datetime.strptime(end_date, "%Y-%m-%d")

    date_list = []
    pool_count_list = []
    currency_pairs_list = []

    while current_date <= end_date:
        formatted_date = current_date.strftime("%Y-%m-%d")
        if formatted_date in processed_dates:
            print(f"Skipping {formatted_date}, already processed.")
        else:
            print(f"Fetching data for {formatted_date}...")
            block_number = get_block_number_for_date(formatted_date)
            if block_number is None:
                print(f"Error: Unable to fetch block number for {formatted_date}")
                date_list.append(formatted_date)
                pool_count_list.append(None)
                currency_pairs_list.append(None)
            else:
                query = query_template.format(block_number=block_number)
                response = requests.post(url, json={'query': query})
                if response.status_code == 200:
                    data = response.json()
                    pool_count = int(data['data']['uniswapFactory']['pairCount'])
                    pairs = data['data']['pairs']
                    currency_pairs = [f"{pair['token0']['symbol']}/{pair['token1']['symbol']}" for pair in pairs]

                    # Append data to lists
                    date_list.append(formatted_date)
                    pool_count_list.append(pool_count)
                    currency_pairs_list.append(", ".join(currency_pairs))

                    # Save to CSV after each successful fetch
                    new_data = pd.DataFrame({
                        'date': [formatted_date],
                        'pool_count': [pool_count],
                        'currency_pairs': [", ".join(currency_pairs)]
                    })
                    new_data.to_csv(output_file, mode='a', header=not os.path.exists(output_file), index=False)
                else:
                    print(f"Error: {response.status_code}, unable to fetch data for {formatted_date}")
                    date_list.append(formatted_date)
                    pool_count_list.append(None)
                    currency_pairs_list.append(None)

            time.sleep(1)  # To respect rate limits

        current_date += datetime.timedelta(days=1)

    # Combine existing and new data
    if date_list:
        new_data = pd.DataFrame({
            'date': date_list,
            'pool_count': pool_count_list,
            'currency_pairs': currency_pairs_list
        })
        combined_data = pd.concat([existing_data, new_data], ignore_index=True).drop_duplicates(subset=['date'])
        combined_data.to_csv(output_file, index=False)
        return combined_data
    else:
        return existing_data

def get_block_number_for_date(date_str):
    """
    Fetch the Ethereum block number for a given date.
    :param date_str: Date in YYYY-MM-DD format
    :return: Block number corresponding to the date
    """
    date = datetime.datetime.strptime(date_str, "%Y-%m-%d")
    timestamp = int(date.timestamp())
    # Etherscan API URL
    etherscan_url = f"https://api.etherscan.io/api?module=block&action=getblocknobytime&timestamp={timestamp}&closest=before&apikey=YourEtherscanAPIKey"
    response = requests.get(etherscan_url)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == '1':
            return int(data['result'])
    return None

if __name__ == "__main__":
    start_date = "2020-05-18"
    end_date = datetime.datetime.today().strftime("%Y-%m-%d")
    api_key = "YOUR_THE_GRAPH_API_KEY"  # Replace with your The Graph API key

    df = fetch_uniswap_v2_pools(start_date, end_date, api_key)
    print(df)

Skipping 2020-05-18, already processed.
Skipping 2020-05-19, already processed.
Skipping 2020-05-20, already processed.
Skipping 2020-05-21, already processed.
Skipping 2020-05-22, already processed.
Skipping 2020-05-23, already processed.
Skipping 2020-05-24, already processed.
Skipping 2020-05-25, already processed.
Skipping 2020-05-26, already processed.
Skipping 2020-05-27, already processed.
Skipping 2020-05-28, already processed.
Skipping 2020-05-29, already processed.
Skipping 2020-05-30, already processed.
Skipping 2020-05-31, already processed.
Skipping 2020-06-01, already processed.
Skipping 2020-06-02, already processed.
Skipping 2020-06-03, already processed.
Skipping 2020-06-04, already processed.
Skipping 2020-06-05, already processed.
Skipping 2020-06-06, already processed.
Skipping 2020-06-07, already processed.
Skipping 2020-06-08, already processed.
Skipping 2020-06-09, already processed.
Skipping 2020-06-10, already processed.
Skipping 2020-06-11, already processed.


Fetching data for 2022-07-21...
Fetching data for 2022-07-22...
Fetching data for 2022-07-23...
Fetching data for 2022-07-24...
Fetching data for 2022-07-25...
Fetching data for 2022-07-26...
Fetching data for 2022-07-27...
Fetching data for 2022-07-28...
Fetching data for 2022-07-29...
Fetching data for 2022-07-30...
Fetching data for 2022-07-31...
Fetching data for 2022-08-01...
Fetching data for 2022-08-02...
Fetching data for 2022-08-03...
Fetching data for 2022-08-04...
Fetching data for 2022-08-05...
Fetching data for 2022-08-06...
Fetching data for 2022-08-07...
Fetching data for 2022-08-08...
Fetching data for 2022-08-09...
Fetching data for 2022-08-10...
Fetching data for 2022-08-11...
Fetching data for 2022-08-12...
Fetching data for 2022-08-13...
Fetching data for 2022-08-14...
Fetching data for 2022-08-15...
Fetching data for 2022-08-16...
Fetching data for 2022-08-17...
Fetching data for 2022-08-18...
Fetching data for 2022-08-19...
Fetching data for 2022-08-20...
Fetching

Fetching data for 2023-04-04...
Fetching data for 2023-04-05...
Fetching data for 2023-04-06...
Fetching data for 2023-04-07...
Fetching data for 2023-04-08...
Fetching data for 2023-04-09...
Fetching data for 2023-04-10...
Fetching data for 2023-04-11...
Fetching data for 2023-04-12...
Fetching data for 2023-04-13...
Fetching data for 2023-04-14...
Fetching data for 2023-04-15...
Fetching data for 2023-04-16...
Fetching data for 2023-04-17...
Fetching data for 2023-04-18...
Fetching data for 2023-04-19...
Fetching data for 2023-04-20...
Fetching data for 2023-04-21...
Fetching data for 2023-04-22...
Fetching data for 2023-04-23...
Fetching data for 2023-04-24...
Fetching data for 2023-04-25...
Fetching data for 2023-04-26...
Fetching data for 2023-04-27...
Fetching data for 2023-04-28...
Fetching data for 2023-04-29...
Fetching data for 2023-04-30...
Fetching data for 2023-05-01...
Fetching data for 2023-05-02...
Fetching data for 2023-05-03...
Fetching data for 2023-05-04...
Fetching

Fetching data for 2023-12-17...
Fetching data for 2023-12-18...
Fetching data for 2023-12-19...
Fetching data for 2023-12-20...
Fetching data for 2023-12-21...
Fetching data for 2023-12-22...
Fetching data for 2023-12-23...
Fetching data for 2023-12-24...
Fetching data for 2023-12-25...
Fetching data for 2023-12-26...
Fetching data for 2023-12-27...
Fetching data for 2023-12-28...
Fetching data for 2023-12-29...
Fetching data for 2023-12-30...
Fetching data for 2023-12-31...
Fetching data for 2024-01-01...
Fetching data for 2024-01-02...
Fetching data for 2024-01-03...
Fetching data for 2024-01-04...
Fetching data for 2024-01-05...
Fetching data for 2024-01-06...
Fetching data for 2024-01-07...
Fetching data for 2024-01-08...
Fetching data for 2024-01-09...
Fetching data for 2024-01-10...
Fetching data for 2024-01-11...
Fetching data for 2024-01-12...
Fetching data for 2024-01-13...
Fetching data for 2024-01-14...
Fetching data for 2024-01-15...
Fetching data for 2024-01-16...
Fetching

Fetching data for 2024-08-30...
Fetching data for 2024-08-31...
Fetching data for 2024-09-01...
Fetching data for 2024-09-02...
Fetching data for 2024-09-03...
Fetching data for 2024-09-04...
Fetching data for 2024-09-05...
Fetching data for 2024-09-06...
Fetching data for 2024-09-07...
Fetching data for 2024-09-08...
Fetching data for 2024-09-09...
Fetching data for 2024-09-10...
Fetching data for 2024-09-11...
Fetching data for 2024-09-12...
Fetching data for 2024-09-13...
Fetching data for 2024-09-14...
Fetching data for 2024-09-15...
Fetching data for 2024-09-16...
Fetching data for 2024-09-17...
Fetching data for 2024-09-18...
Fetching data for 2024-09-19...
Fetching data for 2024-09-20...
Fetching data for 2024-09-21...
Fetching data for 2024-09-22...
Fetching data for 2024-09-23...
Fetching data for 2024-09-24...
Fetching data for 2024-09-25...
Fetching data for 2024-09-26...
Fetching data for 2024-09-27...
Fetching data for 2024-09-28...
Fetching data for 2024-09-29...
Fetching